In [ ]:
# Importar as bibliotecas
import pandas as pd 
import numpy as np

ENDERECO_DADOS = 'https://www.ispdados.rj.gov.br/Arquivos/BaseDPEvolucaoMensalCisp.csv'

# Obtendo dados
try:
    df_roubos = pd.read_csv(ENDERECO_DADOS, sep=';', encoding='iso-8859-1')
    # print(df_roubos['munic'].unique())  # testar as impressões
    print(df_roubos['regiao'].unique())  # testar as impressões
    print(df_roubos.head())
except Exception as e:
    print("Erro ao obter dados do ISP: ", e)


In [ ]:
# Tratando e preparando os dados
try:
    # Delimitando as variáveis
    df_roubos = df_roubos[['cisp', 'regiao', 'munic', 'roubo_veiculo']]
    # print(df_roubos.head())
    # Tratando os erros de acentuação na série região
    correcao_acentuacao = df_roubos['regiao'].str.startswith('Grande Niter', na=False)
    # print(correcao_acentuacao)
    df_roubos.loc[correcao_acentuacao, 'regiao'] = 'Grande Niterói'
    # print(df_roubos['regiao'].unique())

    # Agrupando os dados cisp, região e município
    df_roubos = df_roubos.groupby(['cisp', 'regiao', 'munic']).sum(['roubo_veiculo']).reset_index()

    # Display() é uma função do Jupyter Notebook que permite exibir o DataFrame
    display(df_roubos)
except Exception as e:
    print("Erro no tratamento dos dados: ", e)

In [ ]:
# Calculando as medidas
try:
    array_roubos = np.array(df_roubos['roubo_veiculo'])
    # print(array_roubos)
    
    media = np.mean(array_roubos)
    mediana = np.median(array_roubos)
    total = np.sum(array_roubos)
    # maximo = np.max(array_roubos)
    # minimo = np.min(array_roubos)

    # Obtendo os Quartis
    q1 = np.quantile(array_roubos, 0.25)
    q3 = np.quantile(array_roubos, 0.75)

    # Printando as medidas
    print('Medidas:')
    print(f'Media: {media:.2f}')
    print(f'Mediana: {mediana}')
    print(f'Total: {total}')

except Exception as e:
    print("Erro ao obter as medidas: ", e)

In [ ]:
# Identificando os maiores e menores
try:
    # Gerar um dataframe com os maiores
    # Copy() quando preciar alterar um dataframe já filtrado
    df_maiores =  df_roubos[df_roubos['roubo_veiculo'] > q3].copy()
    df_maiores['flag'] = 'mais'
    # print(df_maiores)

    # Gerar um dataframe com os menores
    df_menores = df_roubos[df_roubos['roubo_veiculo'] < q1].copy()
    df_menores['flag'] = 'menos'
    # print(df_menores)

    # Concatenar os dois dataframes
    df_roubos_flags = pd.concat([df_maiores, df_menores], ignore_index=True)

    display(df_roubos_flags)
except Exception as e:
    print("Erro ao identificar os maiores e menores: ", e)

In [18]:
# Exportando dados csv ou xlsx
try:
    # Testar sem encoding | Impirmir com o encoding utf-8-sig, se precisar do csv fora do BI
    df_roubos_flags.to_csv('roubos_veiculos.csv', index=False, )

    # df_roubos_flags.to_excel('roubos_veiculos.xlsx', index=False)

except Exception as e:
    print("Erro ao exportar os dados: ", e)


In [19]:
# CRUD
# Create, Read, Update, Delete
# Create - Criar um novo registro
# Read - Ler os dados existentes
# Update - Atualizar um registro existente
# Delete - Excluir um registro existente

In [20]:
# Jogar o dataframe como uma tabela no Banco de dados
from sqlalchemy import create_engine, text

try:
    host = 'localhost'
    user = 'root'
    password = ''
    database = 'bd_roubos_flags'

    engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}/{database}')

    tabela = 'tb_roubo_veiculos'
    # CREATE
    # Na primeira vez, usar o 'replace' ou 'fail' para criar a tabela
    # 'replace' irá substituir a tabela se ela já existir, enquanto 'fail' irá falhar se a tabela já existir.
    # A partir da segunda vez, usar o 'append' para adicionar os dados.
    df_roubos_flags.to_sql(tabela, con=engine, if_exists='fail', index=False)

except Exception as e:
    print("Erro ao exportar os dados: ", e)

Erro ao exportar os dados:  Table 'tb_roubo_veiculos' already exists.


In [ ]:
# READ
# Estrutura with para abrir e fechar o arquivo
with engine.connect() as conexao:
    comando_sql = text("SELECT * FROM tb_roubo_veiculos")
    resultado = conexao.execute(comando_sql)
    for linha in resultado:
        print(linha)

In [23]:
# CREATE
with engine.connect() as conexao:
    comando_sql = text(
        """
            INSERT INTO tb_roubo_veiculos (cisp, regiao, munic, roubo_veiculo, flag)
            VALUES (171, 'Grande Niterói', 'São Gonçalo', 500, 'mais')
        """
    )
    
    conexao.execute(comando_sql)

    conexao.commit()

In [26]:
# CREATE
# Estrutura with para abrir e fechar o arquivo
with engine.connect() as conexao:
    comando_sql = text(
    # Inserir a 69 também ('Niterói', 'Grande Niterói', 'mais')
        """
        INSERT INTO tb_roubo_veiculos (cisp, regiao, munic, roubo_veiculo, flag)
        VALUES (:cisp, :regiao, :munic, :roubo_veiculo, :flag)
        """
    )
    
    conexao.execute(comando_sql, {
        'cisp': 69,
        'regiao': 'Grande Niterói',
        'munic': 'São Gonçalo',
        'roubo_veiculo': 100,
        'flag': 'menos'
    })

    conexao.commit()


In [25]:
# UPDATE
# Onde a cisp é 171, passa a ser 76,
# e atualizando para São Gonçalo, flag 'menos', roubo 20 etc
with engine.connect() as conexao:

    regiao = 'Grande Niterói'
    munic = 'Niterói'
    roubo_veiculo = 20
    flag = 'menos'
    cisp_nova = 76
    cisp_atual = 171

    comando_sql = text(
        """
            UPDATE tb_roubo_veiculos
            SET regiao = :regiao,
                cisp = :cisp_nova,
                munic = :munic,
                roubo_veiculo = :roubo_veiculo,
                flag = :flag
            WHERE cisp = :cisp_atual
        """
    )
    # DADOS A SEREM ENVIADOS PARA O BANCO DE DADOS
    conexao.execute(comando_sql, {
        'regiao': regiao,
        'cisp_nova': cisp_nova,
        'munic': munic,
        'roubo_veiculo': roubo_veiculo,
        'flag': flag,
        'cisp_atual': cisp_atual
        }
    )
    
    conexao.commit()

In [28]:
 # DELETE
with engine.connect() as conexao:

    comando_sql = text(
        """
            DELETE FROM tb_roubo_veiculos
            WHERE cisp = :cisp
        """
    )

    conexao.execute(comando_sql, {
        'cisp': 76
        }
    )

    conexao.commit()

    # Sugestão:
    # Com engine.connect(), normalmente é necessário o .commit()
    # Usar engine.begin() no lugar de engine.connect(), 
    # para que o commit seja automático: